In [3]:
from yfpy.query import YahooFantasySportsQuery
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [110]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427
query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET,
)

In [22]:
league_info = query.get_league_info()
start_week = league_info.start_week
end_week = league_info.end_week

In [110]:
def get_team_stats_by_date(team_id, date):
    team_key = f"{league_info.league_key}.t.{team_id}"
    return query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/team/{team_key}/stats;type=date;date={date}", 
        ["team", ["team_points", "team_projected_points"]]
    )

In [79]:
def get_players(player_keys):
    return [
        player 
        for i in range(round(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [54]:
def get_top_n_players_by_position(n, position):
    if position == "F":
        position = "C,LW,RW"
    return [playere for i in range(round(n/25)+1) for player in query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;sort=PTS;sort_type=season;position={position};count={n};start={i*25}/stats",
        ["league", "players"]
    )]

In [125]:
'''
Biggest draft busts/steals
'''
# Get draft results
draft_results = query.get_league_draft_results()
draft_players = get_players([draft_result.player_key for draft_result in draft_results])
draft_players_by_pos = {"F": [], "D": [], "G": []}
positions_map = {"C": "F", "LW": "F", "RW": "F", "D": "D", "G": "G"}
for draft_player in draft_players:
    draft_players_by_pos[positions_map[draft_player.primary_position]].append(draft_player)
# Get the top players by position
top_players_by_pos = {
    "F": get_top_n_players_by_position(len(draft_players_by_pos["F"]), "F"), 
    "D": get_top_n_players_by_position(len(draft_players_by_pos["D"]), "D"), 
    "G": get_top_n_players_by_position(len(draft_players_by_pos["G"]), "G")
}
# Get differences between draft player and top player
diffs = []
for key in draft_players_by_pos:
    for draft_player, top_player in zip(draft_players_by_pos[key], top_players_by_pos[key]):
        diff = round(draft_player.player_points.total - top_player.player_points.total, 1)
        diffs.append((diff, draft_player))
biggest_diffs = sorted(diffs, reverse=True, key=lambda x: x[0])
smallest_diffs = sorted(diffs, key=lambda x: x[0])
print("Biggest draft steals")
for diff, player in biggest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")
print("Biggest draft busts")
for diff, player in smallest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")

Biggest draft busts
J.T. Miller - 97.6
Evan Bouchard - 76.6
Nathan MacKinnon - 76.3
Noah Dobson - 73.2
Thatcher Demko - 62.0
Biggest draft steals
Dougie Hamilton - -293.2
Tyson Barrie - -225.8
Mikhail Sergachev - -213.2
Jack Hughes - -146.6
Devon Levi - -144.0


In [ ]:
'''
Biggest comeback
Time: 5m 49.7s
Requests: 1176
'''
req_count = 0
max_deficit = 0
max_deficit_matchup = None
for week_num in range(start_week, end_week + 1):
    matchups = query.get_league_matchups_by_week(week_num)
    req_count += 1
    for matchup in matchups:
        # Comeback win can't happen without a winner
        if matchiup.is_tied:
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_id, team_l_id = (team_ids[0], team_ids[1]) if winner_id == team_ids[0] else (team_ids[1], team_ids[0])
        deficit = 0
        start_date = datetime.strptime(matchup.week_start, "%Y-%m-%d")
        end_date = datetime.strptime(matchup.week_end, "%Y-%m-%d")
        # Don't include the last day since the matchup is over
        for i in range((end_date - start_date).days):
            current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
            team_w_points = get_team_stats_by_date(team_w_id, current_date_str).team_points.total
            team_l_points = get_team_stats_by_date(team_l_id, current_date_str).team_points.total
            req_count += 2
            deficit = round(deficit + team_l_points - team_w_points, 1)
            if deficit > max_deficit:
                max_deficit = deficit
                max_deficit_matchup = matchup

In [ ]:
print(max_deficit, max_deficit_matchup)

In [ ]:
req_count



# Unsused Code

In [117]:
def get_league_matchups_by_date(date):
    return query.query(f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/scoreboard;date={date}", ["league", "scoreboard", "0", "matchups"])

In [ ]:
'''
Biggest comeback, doesn't work
'''
# TODO: Consider edge case where tied
# Convert strings to datetime objects
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

# TODO: Don't include the last day since the matchup is over
req_count = 1
max_deficit = 0
max_deficit_matchup = None
deficits = [0] * 4 # TODO: Dynamically calculate num_matchups
for i in range((end_date - start_date).days):
    current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
    matchups = get_league_matchups_by_date(current_date_str) # This endpoint doesn't work
    req_count += 1
    for j, matchup in enumerate(matchups):
        print(current_date_str, matchup.week_end, current_date_str == matchup.week_end)
        if current_date_str == matchup.week_end:
            deficits = [0] * 4
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_id, team_l_id = (team_ids[0], team_ids[1]) if winner_id == team_ids[0] else (team_ids[1], team_ids[0])
        team_l_points = matchup.teams[0].team_points.total if winner_id == matchup.teams[0] else matchup.teams[1].team_points.total
        team_w_points = matchup.teams[1].team_points.total if winner_id == matchup.teams[1] else matchup.teams[0].team_points.total
        deficits[j] = round(deficits[j] + team_l_points - team_w_points, 1)
    if max(deficits) > max_deficit:
        max_deficit = max(deficits)
        max_meficit_matchup_index = deficits.index(max_deficit)
        max_deficit_matchup = matchups[max_meficit_matchup_index]